In [1]:
#Geo
#from geopy.geocoders import Nominatim

#Postal Code
from pypostalcode import PostalCodeDatabase

#Tables
import pandas as pd

#Regex
import re

#Deepparse
from deepparse.parser import AddressParser #long import

# Clean Organization Data

In [98]:
dfOrg = pd.read_excel('Organizations.xlsx').iloc[:,1:]

In [3]:
dfOrg.columns

Index(['ID', '2E CONTACT POUR', 'ADDRESS', 'AVIS', 'CONTACT', 'CONTRIBUTION',
       'COURRIEL', 'DDD', 'DOMAINE DINTERET', 'FAF', 'FAX', 'FILIALE DE',
       'ISFOUNDATION', 'LANGUE', 'LIMITES GEOG', 'N DE TEL', 'NAME',
       'NBRE DE SUCC', 'NOMBRE DEMPLOYES', 'NOTE', 'POSTE', 'PRINCIP FILIALES',
       'SECTEUR INDUSTRIEL', 'SITE WEB'],
      dtype='object')

In [64]:
def cleanUpSpaces(row_value):
    """
    Function to remove excess whitespaces
    """
    return ' '.join(row_value.split())

In [ ]:
#Clean the spaces for address
dfOrg.ADDRESS = dfOrg.ADDRESS.apply(lambda x: cleanUpSpaces(x))

# Address

## Deep Learning Approach

In [7]:
address_parser = AddressParser(model_type="bpemb", device=0) #Device = 0 means set to GPU, needs google colab

C:\Users\louis\anaconda3\envs\Scraping\lib\site-packages\deepparse\parser\address_parser.py:881: UserWarning: No CUDA device detected, device will be set to 'CPU'.
  warnings.warn("No CUDA device detected, device will be set to 'CPU'.")


Loading the embeddings model
downloading https://nlp.h-its.org/bpemb/multi/multi.wiki.bpe.vs100000.model


100%|████████████████████████████████████████████████████████████████████| 1965223/1965223 [00:00<00:00, 2481675.94B/s]


downloading https://nlp.h-its.org/bpemb/multi/multi.wiki.bpe.vs100000.d300.w2v.bin.tar.gz


100%|███████████████████████████████████████████████████████████████| 112202964/112202964 [00:10<00:00, 11095999.52B/s]
C:\Users\louis\anaconda3\envs\Scraping\lib\site-packages\deepparse\network\seq2seq.py:99: UserWarning: No pre-trained model where found in the cache directory C:\Users\louis\.cache\deepparse. Thus, we willautomatically download the pre-trained model.
  warnings.warn(


In [8]:
parsed_addresses = address_parser(dfOrg.ADDRESS)


Vectorizing the address


In [39]:
fields = ['StreetNumber', 'StreetName', 'Municipality', 'Province', 'PostalCode']
df_parsed_address = pd.DataFrame([parsed_address.to_dict(fields=fields) for parsed_address in parsed_addresses],
                                         columns=fields)

In [40]:
df_parsed_address['Province'] = df_parsed_address['Province'].str.upper()
df_parsed_address['PostalCode'] = df_parsed_address['PostalCode'].str.upper()

In [42]:
df_parsed_address

,StreetNumber,StreetName,Municipality,Province,PostalCode
0,7290,rue frederick,banting saint-laurent,QC,H4T1Z2
1,305,12e avenue,richelieu,QC,J3L3T2
2,3750,chemin de,l'aéroport trois-rivières,QC,G9B2N8
3,800,boul. hymes,saint-laurent,QC,H4S0B5
4,800,boul. hymes,saint-laurent,QC,H4S0B5
...,...,...,...,...,...
1337,3400,boul. de maisonneuve,montréal,QC,H3Z3G1
1338,1301,rue ampère,boucherville,QC,J4B5Z5
1339,40,rue du,moulin warwick,QC,J0A1M0
1340,860,boul. gérard-cadieux,salaberry-de-valleyfield,QC,J6T6L4


In [48]:
addressDicts = df_parsed_address.to_dict('records')

In [49]:
addressDicts

[{'StreetNumber': '7290',
  'StreetName': 'rue frederick',
  'Municipality': 'banting saint-laurent',
  'Province': 'QC',
  'PostalCode': 'H4T1Z2'},
 {'StreetNumber': '305',
  'StreetName': '12e avenue',
  'Municipality': 'richelieu',
  'Province': 'QC',
  'PostalCode': 'J3L3T2'},
 {'StreetNumber': '3750',
  'StreetName': 'chemin de',
  'Municipality': "l'aéroport trois-rivières",
  'Province': 'QC',
  'PostalCode': 'G9B2N8'},
 {'StreetNumber': '800',
  'StreetName': 'boul. hymes',
  'Municipality': 'saint-laurent',
  'Province': 'QC',
  'PostalCode': 'H4S0B5'},
 {'StreetNumber': '800',
  'StreetName': 'boul. hymes',
  'Municipality': 'saint-laurent',
  'Province': 'QC',
  'PostalCode': 'H4S0B5'},
 {'StreetNumber': '8401',
  'StreetName': 'rte transcanadienne',
  'Municipality': 'saint-laurent',
  'Province': 'QUÉBEC',
  'PostalCode': 'H4S1Z12'},
 {'StreetNumber': '7950',
  'StreetName': 'vauban',
  'Municipality': 'montréal',
  'Province': 'QC',
  'PostalCode': 'H1J2X5'},
 {'StreetNum

In [50]:
#Print real address and parsed in order to compare
for i, df_address in enumerate(dfOrg.ADDRESS):
    print(df_address)
    temp = addressDicts[i]
    print(f"{temp['StreetNumber']}, {temp['StreetName']}, {temp['Municipality']}, {temp['Province']}, {temp['PostalCode']}")
    print("\n")

7290, rue Frederick Banting Saint-Laurent QC H4T1Z2
7290, rue frederick, banting saint-laurent, QC, H4T1Z2


305, 12e Avenue Richelieu QC J3L3T2
305, 12e avenue, richelieu, QC, J3L3T2


3750, chemin de l'aéroport Trois-Rivières QC G9B2N8
3750, chemin de, l'aéroport trois-rivières, QC, G9B2N8


800 boul. Hymes Saint-Laurent QC H4S0B5
800, boul. hymes, saint-laurent, QC, H4S0B5


800 boul. Hymes Saint-Laurent QC H4S0B5
800, boul. hymes, saint-laurent, QC, H4S0B5


8401, rte Transcanadienne Saint-Laurent Québec H4S1Z12
8401, rte transcanadienne, saint-laurent, QUÉBEC, H4S1Z12


7950, Vauban Montréal QC H1J2X5
7950, vauban, montréal, QC, H1J2X5


75, rue Queen, bur. 6100 Montréal QC H3C2N6
75, rue queen, montréal, QC, H3C2N6


75, rue Queen, bur. 6100 Montréal QC H3C2N6
75, rue queen, montréal, QC, H3C2N6


2270, rue Garneau Longueuil QC J4G1E7
2270, rue garneau, longueuil, QC, J4G1E7


1331, rue Graham-Bell Boucherville QC J4B6A1
1331, rue graham-bell, boucherville, QC, J4B6A1


3000, rue



600, boul. de Maisonneuve O., bur. 500 Montréal QC H3A3J2
600, boul. de maisonneuve, montréal, QC, H3A3J2


20, Place du Commerce Îles-des-Soeurs, Verdun QC H3E1Z6
20, place du commerce, îles-des-soeurs verdun, QC, H3E1Z6


160, boul. Industriel Rouyn-Noranda QC J9X6T3
160, boul. industriel, rouyn-noranda, QC, J9X6T3


5, rue Industrielle Saint-Roch-de-l'Achigan QC J0K3H0
5, rue industrielle, saint-roch-de-l'achigan, QC, J0K3H0


1250, rue Nobel, bur. 200 Boucherville QC J4B5H1
1250, rue nobel bur., boucherville, QC, J4B5H1


9455, rue J.-J. Gagnier Montréal QC H1Z3C8
9455, rue j.-j., gagnier montréal, QC, H1Z3C8


9880, ave de l'Esplanade Montréal QC H3L2X5
9880, ave de, l'esplanade montréal, QC, H3L2X5


1570, rue Ampère, bur. 403 Boucherville QC J4B7L4
1570, rue ampère bur., boucherville, QC, J4B7L4


8200, boul. Décarie, bur. 200 Montréal QC H4P2P5
8200, boul. décarie, montréal, QC, H4P2P5


1010, rue Sherbrooke O., 20e étage Montréal QC H3A2R7
1010, rue sherbrooke o., montréal, 

3521, boul. Thimens Saint-Laurent QC H4R1V5
3521, boul. thimens, saint-laurent, QC, H4R1V5


22 Aerial Place N.E. Calgary AB T2E3J1
22, aerial place, n.e. calgary, AB, T2E3J1


285, rue St-Louis Warwick QC J0A1M0
285, rue st-louis, warwick, QC, J0A1M0


21 919, rue Dumberry Vaudreuil-Dorion QC J7V8P7
21, 919 rue, dumberry vaudreuil-dorion, QC, J7V8P7


1600, boul. René-Lévesque O., 11e étage Montréal QC H3H1P9
1600, boul. rené-lévesque o., montréal, QC, H3H1P9


3400, boul. de Maisonneuve O., bur. 900 Montréal QC H3Z3G1
3400, boul. de maisonneuve, montréal, QC, H3Z3G1


1301, rue Ampère Boucherville QC J4B5Z5
1301, rue ampère, boucherville, QC, J4B5Z5


40, rue du Moulin Warwick QC J0A1M0
40, rue du, moulin warwick, QC, J0A1M0


860, boul. Gérard-Cadieux Salaberry-de-Valleyfield QC J6T6L4
860, boul. gérard-cadieux, salaberry-de-valleyfield, QC, J6T6L4


1100, boul. René-Lévesque O., bur. 1840 Montréal QC H3B4N4
1100, boul. rené-lévesque o., montréal, QC, H3B4N4




In [57]:
dfOrg = pd.concat([dfOrg, df_parsed_address], axis=1)

In [58]:
dfOrg

,ID,2E CONTACT POUR,ADDRESS,AVIS,CONTACT,CONTRIBUTION,COURRIEL,DDD,DOMAINE DINTERET,FAF,...,NOTE,POSTE,PRINCIP FILIALES,SECTEUR INDUSTRIEL,SITE WEB,StreetNumber,StreetName,Municipality,Province,PostalCode
0,1,NaN,"7290, rue Frederick Banting Saint-Laurent QC ...",Dons autres qu'en argent. Dons en « temps » (b...,"Madame Leanne Hayward, Superviseur aux communi...",Dons & commandite,lhayward@mmm.com,En tout temps,"Éducation, environnement, santé et bien-être ...",31 décembre,...,Toute demande de dons ou de commandites doit ê...,2390,NaN,Équipement et services industriels; Fabricatio...,https://www.3mcanada.ca/,7290,rue frederick,banting saint-laurent,QC,H4T1Z2
1,2,"DonsMadame Marie-Josée Dery, Coordonnatrice Ca...","305, 12e Avenue Richelieu QC J3L3T2",Faire les demandes par : courrier.,"Monsieur Claude Cardin, Président",Dons,mjdery@prevost-architectural.com; info@prevos...,En tout temps,"Cancer, hôpitaux, maladies du coeur et santé.",31 décembre,...,NaN,NaN,NaN,"Matériaux de construction, manufacturiers; Fab...",www.adprevost.ca,305,12e avenue,richelieu,QC,J3L3T2
2,3,NaN,"3750, chemin de l'aéroport Trois-Rivières QC...",Faire les demandes par : courriel.,"Madame Elizabeth Vautour, Adjointe exécutive a...",Dons,elizabeth.vautour@aarcorp.com; comptabilite@aa...,En tout temps,"Éducation, Santé, Bien-être, Vétérans.",31 décembre,...,Envoyer votre demande de dons par courriel à ...,3201,NaN,"Offre des services d'entretien, de réparation ...",https://www.aarcorp.com,3750,chemin de,l'aéroport trois-rivières,QC,G9B2N8
3,4,NaN,800 boul. Hymes Saint-Laurent QC H4S0B5,"Faire les demandes par : courrier, fax.","Madame Carley Tietolman, Responsable des dons ...",Dons & commandite,NaN,En tout temps,Jeunesse (local seulement).,NaN,...,Toute demande de dons ou de commandites doit ê...,NaN,NaN,Équipement et services industriels; Fabriquer ...,http://new.abb.com/ca,800,boul. hymes,saint-laurent,QC,H4S0B5
4,5,NaN,800 boul. Hymes Saint-Laurent QC H4S0B5,Faire les demandes par : courrier.,"Monsieur Guido Mussehl, Contrôleur Finances",Dons & commandite,guido.mussehl@ca.abb.comMadame Nathalie Arsena...,En tout temps,"Centraide, développement économique et social...",31 décembre,...,Par courrier seulement.,NaN,NaN,Commerce de gros et détail; Grossistes-distrib...,www.ABB.com/ca,800,boul. hymes,saint-laurent,QC,H4S0B5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337,1338,NaN,"3400, boul. de Maisonneuve O., bur. 900 Montr...",Dons en « temps » (bénévolat). Faire les deman...,"Madame Cynthia Laberge, Adjointe Marketing",Dons,cynthia.laberge@xerox.com,En tout temps,"Appui aux org. communautaires, éducation, peup...",31 décembre,...,Sur le site web sélectionner : À propos de Xe...,NaN,"Xerox Canada, Société de financement Xerox; Xe...","Commerce de gros et détail; Équipement, systèm...",https://www.xerox.ca,3400,boul. de maisonneuve,montréal,QC,H3Z3G1
1338,1339,NaN,"1301, rue Ampère Boucherville QC J4B5Z5",Dons en « temps » (bénévolat). Faire les deman...,Vice-président Finances,Dons & commandite,NaN,Avril à juin,Engagement auprès des communautés (Cris du Qu...,31 mars,...,Quatre nouveaux organismes sont choisis à chaq...,NaN,NaN,Produits récréatifs; Fabricant d'une gamme de ...,www.yamaha-motor.ca,1301,rue ampère,boucherville,QC,J4B5Z5
1339,1340,NaN,"40, rue du Moulin Warwick QC J0A1M0",Faire les demandes par : courrier.,"Madame Renée-Maude Jalbert, Directeure Marketing",Commandite,renee-maude.jalbert@yum-yum.com,En tout temps,"Causes charitables en général, région des Boi...",NaN,...,Prévoir un délai de 4 semaines pour l'étude d'...,NaN,NaN,Alimentation; Fabricant de croustilles et grig...,www.yum-yum.com,40,rue du,moulin warwick,QC,J0A1M0
1340,1341,NaN,"860, boul. Gérard-Cadieux Salaberry-de-Valley...","Faire les demandes par : courrier, courriel.","Monsieur Sylvain Lirette, Directeur des financ...",Dons & commandite,info.cezinc@glencore-ca.com; info.cezinc@glenc...,En tout temps,"Appuie plusieurs activités d